## Aplicar el modelo CatBoost

In [10]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier


## 1. Cargar modelo CatBoost

In [11]:
modelo = CatBoostClassifier()
modelo.load_model("best_model_catboost.cbm")
print("✅ Modelo CatBoost cargado correctamente")

✅ Modelo CatBoost cargado correctamente


##  2. Crear data ficticia (845 pacientes con errores simulados)

In [12]:
np.random.seed(42)
n = 845

data_ficticia = pd.DataFrame({
    'Age': np.random.randint(28, 85, n),
    'Sex': np.random.choice(['M', 'F', 'Male', 'Female', ''], n),
    'ChestPainType': np.random.choice(['ATA', 'NAP', 'ASY', 'TA', None, ''], n),
    'RestingBP': np.random.choice(list(range(90, 201)) + [0, None], n),
    'Cholesterol': np.random.choice(list(range(100, 351)) + [0, '', None], n),
    'FastingBS': np.random.choice([0, 1, 2, '', None], n),
    'RestingECG': np.random.choice(['Normal', 'ST', 'LVH', '', None], n),
    'MaxHR': np.random.randint(90, 202, n),
    'ExerciseAngina': np.random.choice(['N', 'Y', 'Yes', 'No', None, ''], n),
    'Oldpeak': np.round(np.random.uniform(0, 6.2, n), 1),
    'ST_Slope': np.random.choice(['Up', 'Flat', 'Down', None, ''], n)
})

## 3. Limpieza básica

In [13]:
data_limpia = data_ficticia.copy()

data_limpia['Sex'] = data_limpia['Sex'].replace({'Male': 'M', 'Female': 'F', '': np.nan})
data_limpia['ChestPainType'] = data_limpia['ChestPainType'].replace({'': np.nan})
data_limpia['RestingBP'] = pd.to_numeric(data_limpia['RestingBP'], errors='coerce')
data_limpia['Cholesterol'] = pd.to_numeric(data_limpia['Cholesterol'], errors='coerce')
data_limpia['FastingBS'] = pd.to_numeric(data_limpia['FastingBS'], errors='coerce')
data_limpia['ExerciseAngina'] = data_limpia['ExerciseAngina'].replace({
    'Yes': 'Y', 'No': 'N', '': np.nan
})
data_limpia['RestingECG'] = data_limpia['RestingECG'].replace({'': np.nan})
data_limpia['ST_Slope'] = data_limpia['ST_Slope'].replace({'': np.nan})

# Rellenar nulos hacia adelante (puedes cambiar a .fillna(método='bfill') si prefieres)
data_limpia.fillna(method='ffill', inplace=True)

C:\Users\Pedro Yacila\AppData\Local\Temp\ipykernel_20912\1137943432.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_limpia.fillna(method='ffill', inplace=True)


## 4. Realizar predicción

In [14]:
predicciones = modelo.predict(data_limpia)
data_limpia['Prediccion_HeartDisease'] = predicciones

##  5. Guardar resultados

In [15]:
data_limpia.to_csv("predicciones_845_pacientes.csv", index=False)
print("✅ Archivo 'predicciones_845_pacientes.csv' guardado con éxito.")

✅ Archivo 'predicciones_845_pacientes.csv' guardado con éxito.


In [19]:
# Reemplazar 0 y 1 por etiquetas
data_limpia['Prediccion_HeartDisease'] = data_limpia['Prediccion_HeartDisease'].replace({
    0: "No está enfermo",
    1: "Está enfermo"
})

# Guardar nuevamente
data_limpia.to_csv("predicciones_845_pacientes.csv", index=False)
print("✅ Archivo actualizado con etiquetas descriptivas.")


✅ Archivo actualizado con etiquetas descriptivas.


In [20]:
df = pd.read_csv("predicciones_845_pacientes.csv")

# Mostrar las primeras 5 filas
print(df.head())

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   66   F           NAP       94.0        289.0        NaN     Normal    126   
1   79   F           NAP      172.0        134.0        2.0     Normal    118   
2   56   F           ASY      123.0        246.0        2.0        LVH    156   
3   42   F           ASY       99.0        129.0        0.0        LVH    104   
4   70   F           ASY      178.0        177.0        1.0        LVH    189   

  ExerciseAngina  Oldpeak ST_Slope Prediccion_HeartDisease  
0              N      4.7       Up         No está enfermo  
1              N      3.7       Up         No está enfermo  
2              N      1.2     Flat            Está enfermo  
3              N      4.1       Up            Está enfermo  
4              Y      3.9     Flat            Está enfermo  
